# Concise code for the Kernel Data Challenege


This is a concise code where all the solvers and the kernels we used are implemented without the help of ML library like Scikit Learn. In this code we implement few examples for each kernel and solvers:

1. Gaussian Kernel with Kernel Ridge Regression
2. Spectrum Kernel with Kernel Logistic Regression
3. Mismatch kernel with Kernel SVM

The hyperparameters we used for the examples are not the ones we selected for the submissions. 
In the complete code, we built the three estimators (Ridge, Logistic, SVM) with the solvers of this code, compatible with the scikit learn's API in order to perform efficient cross validation with the GridSearchCV and RandomizedSearchCV functions.
This code only gives a few examples to show that our functions work.
To completely see the amount of work we have done for this project, please see the complete code. 

Thank you and have good reading!

*PS: We use CVXOPT as QP solvers*

## Import of packages 

In [177]:
import numpy as np 
import pandas as pd 
from cvxopt import matrix
from cvxopt import solvers
from itertools import product, compress
from scipy.linalg import solve, lstsq
from itertools import compress, product, combinations
from tqdm import tqdm

## Importation of the preprocessed data 

In [178]:
X_0train = pd.read_csv("Data/Xtr0_mat100.csv", sep=' ', header=None).to_numpy()
X_1train = pd.read_csv("Data/Xtr1_mat100.csv", sep=' ', header=None).to_numpy()
X_2train = pd.read_csv("Data/Xtr2_mat100.csv", sep=' ', header=None).to_numpy()

Y_0 = pd.read_csv("Data/Ytr0.csv", sep=',')
Y_1 = pd.read_csv("Data/Ytr1.csv", sep=',')
Y_2 = pd.read_csv("Data/Ytr2.csv", sep=',')

Y_0train = np.where(Y_0["Bound"]==0, -1, Y_0["Bound"])
Y_1train = np.where(Y_1["Bound"]==0, -1, Y_1["Bound"])
Y_2train = np.where(Y_2["Bound"]==0, -1, Y_2["Bound"])

X_0test = pd.read_csv("Data/Xte0_mat100.csv", sep=' ', header=None).to_numpy()
X_1test = pd.read_csv("Data/Xte1_mat100.csv", sep=' ', header=None).to_numpy()
X_2test = pd.read_csv("Data/Xte2_mat100.csv", sep=' ', header=None).to_numpy()

## Kernel Ridge Regression 

In [179]:
def Kernel_Ridge_Regression(X_train, y_train, lbd=1, weight=False, gamma=1, degree=2, c0=1, k=2, kernel='rbf'):
    if kernel=="rbf":
        K = rbf_kernel(X_train, gamma)
    elif kernel=="poly":
        K = poly_kernel(X_train, X_train, degree, c0)
    elif kernel=="spectrum":
        K = spectrum_kernel(X_train,k)
    elif kernel=="precomputed":
        K = X_train
    n = K.shape[0]
    w = weight

    if isinstance(weight, bool):
        A = (K+n*lbd*np.eye(n))
        alpha = solve(A,y_train,assume_a="sym")
        return alpha

    elif isinstance(weight, str):
            w1 = (y_train==1).mean()
            w0 = 1-w1
            w=np.where(y_train==1, w1, w0)
    wi = (1/w)

    A = K+n*lbd*wi*np.eye(n)
    alpha = solve(A, y_train, assume_a="sym")

    return alpha

## Iterative Reweighted Least Square for Kernel Logistic Regression 

In [180]:
def sigmoid(v):
    return 1 / (1+np.exp(-v))

In [181]:
def IRLS(X_train, y_train, lbd=1, ga=1, degree=2, c0=1, k=2, ker="rbf", n_iter=15, eps=10**-6, method='slow'):
    n = y_train.shape[0]
    if ker=="rbf":
        K = rbf_kernel(X_train, ga)
    elif ker=="poly":
        K = poly_kernel(X_train, X_train, degree, c0)
    elif ker=="spectrum":
        K = spectrum_kernel(X_train, k)
    elif ker=="precomputed":
        K = X_train
    # alpha = Kernel_Ridge_Regression(K, y_train, lbd, False, 1, bs, "precomputed")
    # alpha = np.zeros(n)
    # l = []
    
    alpha=np.zeros(n)
    for i in range(n_iter):
   
        alpha_old = alpha

        m = K.dot(alpha)
        # l.append(log_loss(y_train*m).mean() + lbd*alpha.dot(m))

        p = sigmoid(m)
       
        weight = p*(1-p)
        weight = np.where(weight<0.000001, 0.000001, weight)
        
        u = np.where(sigmoid(y_train*m)<0.000001, 0.000001, sigmoid(y_train*m))
        z = m + y_train/u
    
        S = np.diag(weight**-1)
        A = (K+2*lbd*n*S)
        alpha = solve(A, z, assume_a="sym")

        # print(np.linalg.norm(alpha_old-alpha))

        if np.linalg.norm(alpha_old-alpha) < eps:
            break 
    return alpha # ,l
        

## Kernel SVM

In [1]:
def SVM(X_train, y_train, C=1, gamma=1, degree=2, c0=1, k=2, kernel="rbf"):
    n = y_train.shape[0]
    if kernel=="rbf":
        K = rbf_kernel(X_train, gamma)
    elif kernel=="poly":
        K = poly_kernel(X_train, X_train, degree, c0)
    elif kernel=="spectrum":
        K = spectrum_kernel(X_train, k)
    elif kernel=="precomputed":
        K = X_train
    P = matrix(K, tc='d')
    q = matrix(-y_train, tc='d')
    g1 = np.diag(y_train)
    G = matrix(np.vstack((g1, -g1)), tc='d')
    h = matrix(np.hstack((np.repeat(C,n), np.zeros(n))), tc='d')
    solvers.options['show_progress'] = False
    sol = solvers.qp(P,q,G,h)
    return np.array(sol['x']).reshape(-1,)

## Predictions functions
For all of estimateurs a test point $x$ is affected to class 1 if $f(x)=\sum_{i=1}^{n}\alpha_ik(x,x_i)>0$

In [184]:
def decision_function(alpha, X_test, X_train, kernel, gamma=1, degree=2, c0=1, k=2):
        if kernel=="precomputed":
            # here the matrix k(x_test,x_train) is already computed in X_test
            return X_test.dot(alpha)
       
        elif kernel=="rbf":
            return K_rbf_kernel(X_test, X_train, gamma).dot(alpha) 
        elif kernel=="poly":
            return poly_kernel(X_test, X_train, degree, c0).dot(alpha) 
        elif kernel=="spectrum":
            return K_spectrum_kernel(X_test, X_train, k).dot(alpha) 

def predict(alpha, X_test, X_train, kernel, gamma=1, degree=2, c0=1, k=2):
        scores = decision_function(alpha, X_test, X_train, kernel, gamma, degree, c0, k)
        indices = (scores > 0).astype(np.int)
        return np.array([-1,1])[indices]

## First Example Gaussian kernel with Kernel Ridge

In [185]:
def rbf_kernel(X_train, gamma):
    X_norm = np.sum(X_train ** 2, axis = -1)
    return np.exp(-gamma*(X_norm[:,None]+X_norm[None,:]-2*X_train.dot(X_train.T)))

def K_rbf_kernel(X_test, X_train, gamma):
    Xtt_norm = np.sum(X_test ** 2, axis = -1)
    Xtr_norm = np.sum(X_train ** 2, axis = -1)
    return np.exp(-gamma*(Xtt_norm[:,None]+Xtr_norm[None,:]-2*X_test.dot(X_train.T)))

def poly_kernel(X, Y, degree, c0):
    return (X.dot(Y.T)+c0)**degree

To gain some robustness we standardize the training set. Of course the validation set and the test set will be "standardized" with respect with the train mean and standard deviation. This also allow us to fix gamma of the gaussian kernel to be 1/dimension of features space which is the "natural" value when the features are standardized. This will help when performing cross validation because we will have only one parameter to optimize: the regularization parameter lambda


In [186]:
def Standardize(X, mean=False, sigma=False):
    if isinstance(mean, bool):
        mean = X.mean(axis=0)
        sigma = X.std(axis=0)
        return (X-mean) / sigma, mean, sigma
    else:
        return (X-mean) / sigma
      
def split_train_validation_set(X_train, Y_train, train_size=0.8):
    n_train = int(0.8*Y_train.shape[0])
    idx = np.random.choice(np.arange(Y_train.shape[0]), size=n_train, replace=False)
    if isinstance(X_train, np.ndarray):
        return X_train[idx], Y_train[idx], np.delete(X_train, idx, 0), np.delete(Y_train, idx)
    else:
        return X_train.iloc[idx], Y_train[idx], X_train.drop(list(idx)), np.delete(Y_train, idx)
 

In [187]:
Xtr0, Ytr0, Xva0, Yva0 = split_train_validation_set(X_0train, Y_0train, train_size=0.8)
Xtr0_standardize, mean, sigma = Standardize(Xtr0)

Xva0_standardize = Standardize(Xva0, mean, sigma)
d = 1/X.shape[1]

In [188]:
lam = 1
alpha = Kernel_Ridge_Regression(Xtr0_standardize, Ytr0, lbd=lam, gamma=d, kernel='rbf')

predictions = predict(alpha, Xva0_standardize, Xtr0_standardize, "rbf", gamma=1/d)

accuracy = (predictions==Yva0).mean()
print("The accuracy for Kernel Ridge Regression on this validation set for lambda={}, gaussian kernel with parameter gamma={:.3f}\n is {:.4f}%".format(lam,d,accuracy))
       

The accuracy for Kernel Ridge Regression on this validation set for lambda=1, gaussian kernel with parameter gamma=0.333
 is 0.5700%


# Import of the real data for string kernels

In [189]:
X_0train=pd.read_csv('Data/Xtr0.csv', sep=',')['seq']
X_1train=pd.read_csv('Data/Xtr1.csv', sep=',')['seq']
X_2train=pd.read_csv('Data/Xtr2.csv', sep=',')['seq']

X_0test=pd.read_csv('Data/Xte0.csv', sep=',')['seq']
X_1test=pd.read_csv('Data/Xte1.csv', sep=',')['seq']
X_2test=pd.read_csv('Data/Xte2.csv', sep=',')['seq']

Y_0=pd.read_csv("Data/Ytr0.csv",sep=',')
Y_1=pd.read_csv("Data/Ytr1.csv",sep=',')
Y_2=pd.read_csv("Data/Ytr2.csv",sep=',')

Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])

## Second Examples Spectrum Kernel with Kernel Logistic Regression 

In [190]:
def build_dic_voc(k):
    return {''.join(v):i for i,v in enumerate(product("ACGT",repeat=k))}
    

def phi_X(X,voc):
    x=np.zeros(len(voc))
    for u in X:
        x[voc[u]]+=1
    return x
def spectrum_kernel(X,k):
    voc=build_dic_voc(k)
    phi_x=np.vstack(X.apply(lambda x: [x[i:i+k] for i in range(0, len(x)-k+1)]).apply(phi_X,args=(voc,)).to_numpy())
    return (phi_x.dot(phi_x.T))


def K_spectrum_kernel(X,Y,k):
    voc=build_dic_voc(k)
    phi_x=np.vstack(X.apply(lambda x: [x[i:i+k] for i in range(0, len(x)-k+1)]).apply(phi_X,args=(voc,)).to_numpy())
    phi_y=np.vstack(Y.apply(lambda x: [x[i:i+k] for i in range(0, len(x)-k+1)]).apply(phi_X,args=(voc,)).to_numpy())
    return phi_x.dot(phi_y.T)

In [191]:
Xtr0,Ytr0,Xva0,Yva0=split_train_validation_set(X_0train,Y_0train,train_size=0.8)

In [192]:
lam=1
k=8
alpha=IRLS(Xtr0,Ytr0,lbd=lam,k=8,ker='spectrum')

predictions=predict(alpha,Xva0,Xtr0,"spectrum",k=9)

accuracy=(predictions==Yva0).mean()
print("The accuracy for Kernel Logistic Regression, on this validation set for lambda={}, spectrum kernel with parameter k={}\n is {:.4f}%".format(lam,k,accuracy))

The accuracy for Kernel Logistic Regression, on this validation set for lambda=1, spectrum kernel with parameter k=8
 is 0.6075%


## Mismatch Kernel

In [193]:
def phi_mismatch_X(X,k,m,voc):
    voc1=["A","C","G","T"]
    phi_x=np.zeros(len(voc))
    for d in range(0,len(X)-k+1):
        x=list(X[d:d+k])
        
   
        for mm in (range(m,-1,-1)):
            for i in combinations(range(k),mm):
                list_voc_left=[]
                for y in i:
                    voc_copy=voc1[:]

                    voc_copy.remove(x[y])
                    list_voc_left.append(voc_copy)

                for letters in product(*list_voc_left):
                    x_copy=x[:]
                    for o,p in enumerate(i):
                         x_copy[p]=letters[o]
            
                    yes=''.join(x_copy)
                    
                  
                    phi_x[voc[yes]]= phi_x[voc[yes]]+1
                   
    return phi_x
            
def mismatch_kernel(X,k,m):
    
    voc=build_dic_voc(k)
    phi_x=np.vstack(X.apply(phi_mismatch_X,args=(k,m,voc)).to_numpy())
    return (phi_x.dot(phi_x.T))

def K_mismatch_kernel(X,Y,k,m):
    
    voc=build_dic_voc(k)
    phi_x=np.vstack(X.apply(phi_mismatch_X,args=(k,m,voc)).to_numpy())
    phi_y=np.vstack(Y.apply(phi_mismatch_X,args=(k,m,voc)).to_numpy())
    return (phi_x.dot(phi_y.T))


In [194]:
C=1
k=9
m=1
K_train_train=mismatch_kernel(Xtr0,k,m)
alpha=SVM(K_train_train,Ytr0,C=1,kernel='precomputed')
K_val_train=K_mismatch_kernel(Xva0,Xtr0,9,1)

predictions=predict(alpha,K_val_train,Xtr0,"precomputed")

accuracy=(predictions==Yva0).mean()
print("The accuracy for Kernel SVM, on this validation set for C={}, mismatch kernel with parameter k={}\n and m={} is {:.4f}%".format(lam,k,m,accuracy))

The accuracy for Kernel SVM, on this validation set for C=1, mismatch kernel with parameter k=9
 and m=1 is 0.6250%
